# COMS 4995 HW4
#### Jianfeng Zhuang: JZ3872
#### Huazhang Liu: HL3338

### Task 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import randint

In [12]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.woe import WOEEncoder
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [10]:
wine = pd.read_csv("winemag-data-130k-v2.csv")
wine = wine.drop("Unnamed: 0", axis = 1)
wine.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [11]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


In [37]:
wine_US = wine[wine['country'] == "US"]
wine_US = wine_US.drop('country', axis = 1)
wine_US.index = np.arange(0, wine_US.shape[0])
wine_US

,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
4,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
...,...,...,...,...,...,...,...,...,...,...,...,...
54499,Hailing from one of the more popular vineyards...,Jurassic Park Vineyard Old Vines,90,20.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,Birichino
54500,There's no bones about the use of oak in this ...,Barrel Fermented,90,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs
54501,This opens with herbaceous dollops of thyme an...,Blocks 7 & 22,90,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Hendry 2012 Blocks 7 & 22 Zinfandel (Napa Valley),Zinfandel,Hendry
54502,This Zinfandel from the eastern section of Nap...,NaN,90,22.0,California,Chiles Valley,Napa,Virginie Boone,@vboone,Houdini 2011 Zinfandel (Chiles Valley),Zinfandel,Houdini


From the above, we know that the target column is "points'. 

In [38]:
wine_US['designation'].unique().shape

(14184,)

In [39]:
wine_US[wine_US['designation'] == "Reserve"]

,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
91,"There is an odd, piercing edge to the aromas, ...",Reserve,85,28.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Silvan Ridge 2006 Reserve Pinot Noir (Willamet...,Pinot Noir,Silvan Ridge
98,"Tasty, with pie-filling cherries, blackberries...",Reserve,85,25.0,California,Sonoma Mountain,Sonoma,NaN,NaN,Work 2004 Reserve Merlot (Sonoma Mountain),Merlot,Work
125,"With a fruity-sugary finish, this Zin will app...",Reserve,86,34.0,California,Mendocino,NaN,NaN,NaN,Brutocao 2006 Reserve Zinfandel (Mendocino),Zinfandel,Brutocao
161,"Fine acidity and a bracing minerality, like a ...",Reserve,92,40.0,California,Carneros,Napa-Sonoma,NaN,NaN,Robert Mondavi 2011 Reserve Chardonnay (Carneros),Chardonnay,Robert Mondavi
226,"This is a lovely wine, classically rendered in...",Reserve,93,70.0,California,Howell Mountain,Napa,Virginie Boone,@vboone,W.H. Smith 2012 Reserve Cabernet Sauvignon (Ho...,Cabernet Sauvignon,W.H. Smith
...,...,...,...,...,...,...,...,...,...,...,...,...
54412,"This robust wine is made from 62% Merlot, 37% ...",Reserve,90,140.0,California,Sonoma County-Napa County,Napa-Sonoma,Virginie Boone,@vboone,Pride Mountain 2013 Reserve Claret (Sonoma Cou...,Claret,Pride Mountain
54420,"Blackberry juice mixes with graphite, cedar an...",Reserve,90,50.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Brander 2014 Reserve Cabernet Sauvignon (Santa...,Cabernet Sauvignon,Brander
54462,Dusty tannins make for a soft texture in this ...,Reserve,90,44.0,California,Carneros,Napa-Sonoma,Virginie Boone,@vboone,Ca' Momi 2013 Reserve Merlot (Carneros),Merlot,Ca' Momi
54475,"This wine is mostly Cabernet Sauvignon (78%), ...",Reserve,90,60.0,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Matthews 2012 Reserve Red (Columbia Valley (WA)),Bordeaux-style Red Blend,Matthews


In [58]:
wine_US['region_1'].unique().shape

(265,)

However, since the information contained in this column is complicated, including the city names, county names, even the names of the states, it should be used as text. 

In [56]:
wine_US['region_2'].unique().shape

(18,)

In [61]:
sum(pd.isna(wine_US['region_2']))/len(wine_US)

0.07326067811536768

In [63]:
sum(pd.isna(wine_US['region_2']))

3993

In [53]:
wine_US[wine_US['region_2'] == "Napa"]

,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
3,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
20,This shows jelly-like flavors of orange and pe...,Estate Bottled,85,30.0,California,Napa Valley,Napa,NaN,NaN,RustRidge 2010 Estate Bottled Chardonnay (Napa...,Chardonnay,RustRidge
23,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Okapi 2013 Estate Cabernet Sauvignon (Napa Val...,Cabernet Sauvignon,Okapi
30,"Juicy plum, raspberry and pencil lead lead the...",Bella Vetta Vineyard,86,75.0,California,Howell Mountain,Napa,Virginie Boone,@vboone,Hindsight 2013 Bella Vetta Vineyard Cabernet S...,Cabernet Sauvignon,Hindsight
31,This is the producer's first release of this w...,Estate Grown,86,55.0,California,Calistoga,Napa,Virginie Boone,@vboone,Hindsight 2012 Estate Grown Petite Sirah (Cali...,Petite Sirah,Hindsight
...,...,...,...,...,...,...,...,...,...,...,...,...
54483,"Aged entirely in stainless steel, this combine...",Proprietary,91,30.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Paraduxx 2015 Proprietary White (Napa Valley),White Blend,Paraduxx
54485,Pencil shaving and graphite notes lend a class...,NaN,91,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Conn Creek 2013 Cabernet Sauvignon (Napa Valley),Cabernet Sauvignon,Conn Creek
54500,There's no bones about the use of oak in this ...,Barrel Fermented,90,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs
54501,This opens with herbaceous dollops of thyme an...,Blocks 7 & 22,90,35.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Hendry 2012 Blocks 7 & 22 Zinfandel (Napa Valley),Zinfandel,Hendry


Since there are are total of 14184 unique values of designations, and there are 1485 observations with designation "Reserve", this column is considered as a text column.  

In [42]:
wine_US['taster_name'].unique().shape

(16,)

In [50]:
wine_US['title'].unique().shape

(50229,)

In [49]:
wine_US['variety'].unique().shape

(257,)

In [45]:
wine_US['winery'].unique().shape

(5375,)

Since the column "Winery" consists of 5375 rows, and mostly composited of names, it should be used as a categorical variable through target encoding. 

In [64]:
wine_US_1 = wine_US.drop(['description', 'designation', 'region_1', "taster_twitter_handle", "title"], axis = 1)

In [65]:
wine_US_1.head()

,points,price,province,region_2,taster_name,variety,winery
0,87,14.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Gris,Rainstorm
1,87,13.0,Michigan,NaN,Alexander Peartree,Riesling,St. Julian
2,87,65.0,Oregon,Willamette Valley,Paul Gregutt,Pinot Noir,Sweet Cheeks
3,87,19.0,California,Napa,Virginie Boone,Cabernet Sauvignon,Kirkland Signature
4,87,34.0,California,Sonoma,Virginie Boone,Cabernet Sauvignon,Louis M. Martini
